In [1]:
%load_ext autoreload
%autoreload 2
main_dir = r'f:\Projects\Prvate\Baydin'

data_path = r'E:/Kaggle/05-LabelEncoded-last'
import sys
sys.path.append(main_dir)
from typing import List, Set, Dict, Optional, Any, Tuple, Type, Union

from BayDS.lib.pipeline import *
from BayDS.lib.io import *

import os


## Code to make pipeline node

In [2]:
data = pd.read_pickle(f'{data_path}/label_encoded_data.pkl')

FileNotFoundError: [Errno 2] No such file or directory: 'E:/Kaggle/05-LabelEncoded-last/label_encoded_data.pkl'

In [ ]:
X = data[data.isFraud>=0].drop('isFraud',axis=1)
test = data[data.isFraud<0].drop('isFraud',axis=1)
y= data[data.isFraud>=0].isFraud

In [ ]:
import yaml
categorical_features = yaml.load(open(f'{data_path}/categorical_columns.yaml','r'))
numerical_features = yaml.load(open(f'{data_path}/numerical_columns.yaml','r'))
bad_features = yaml.load(open(f'{data_path}/bad_for_training_features.yaml','r'))

categorical_features = [f for f in categorical_features if f not in bad_features]
all_features =  [f for f in X.columns if f not in bad_features]

In [ ]:
from BayDS.lib.training import *

In [ ]:
drop_big_categorical_features = []
for col in categorical_features:
    if len(data[col].value_counts()) > 50:
        drop_big_categorical_features.append(col)
        print(col, len(data[col].value_counts()))

In [ ]:
# X.drop(features_to_drop,axis=1,inplace=True)
# test.drop(features_to_drop,axis=1,inplace=True)
use_for_train_features =  [f for f in all_features if f not in drop_big_categorical_features]

In [ ]:
categorical_features =  [f for f in categorical_features if f not in drop_big_categorical_features]

In [ ]:
for col in X.columns:
    if str(X[col].dtype) == 'datetime64[ns]':
        print (col, X[col].dtype)
        X[col] = pd.to_timedelta(X[col]).map(lambda qq:qq.days)
        test[col] = pd.to_timedelta(test[col]).map(lambda qq:qq.days)
#         X[col] = X[col].astype(datetime.datetime)

In [ ]:
from sklearn.model_selection import GroupKFold

NFOLDS = 6
folds = GroupKFold(n_splits=NFOLDS)


In [ ]:
params = {
          'objective': 'binary:logistic',
          'max_depth': 10,
          'learning_rate': 0.01,
          "metric": 'auc',
          "verbosity": 1,
          'random_state': 47
         }
train_options = {
        "model_type":'xgb',
        "params": params,
        "eval_metric":'auc',
        'early_stopping_rounds': 500,
        'n_estimators': 10000,
        'averaging': 'usual',
        'use_groups': False,
        'fold_name': folds.__class__.__name__,
        'n_splits': NFOLDS
    }

In [ ]:
result_dict = train_model_classification_vb( splits=1,
                                             X=X, X_test=test, columns=use_for_train_features,
                                             y=y, params=params, folds=folds,
                                             model_type=train_options['model_type'], 
                                             categorial_columns = categorical_features,
                                             plot_feature_importance=True,
                                             verbose=500, early_stopping_rounds=train_options['early_stopping_rounds'],
                                             n_estimators=train_options['n_estimators'], 
                                             averaging=train_options['averaging'],
                                             train_0_sample_coef=0.2, train_1_sample_coef=2.0,
                                             n_jobs=-1, groups=X['DT_M'])


In [26]:
import pickle
pickle.dump(result_dict,open(f'{data_path}/results_dict_xgb-work-1.pkl','wb'))

In [28]:
pd.DataFrame(result_dict['oof'], columns=['isFraud'], index=X.index).to_csv(f'{data_path}/oof_lightgbm-work-1.csv')

pd.DataFrame(result_dict['prediction'], columns=['isFraud'], index=test.index).to_csv(f'{data_path}/prediction_lightgbm-work-1.csv')

sample_submission = pd.read_csv(data_path + '/sample_submission.csv').set_index('TransactionID')

sub1 = pd.DataFrame(result_dict['prediction'], columns=['isFraud'], index=test.index)

sample_submission['isFraud'] = sub1#*0.5 + sub2*0.25 + sub3*0.25

sample_submission.to_csv(f'{data_path}/lightgbm-work-1_sub.csv')

FileNotFoundError: [Errno 2] File b'E:/Kaggle/05-LabelEncoded-last/sample_submission.csv' does not exist: b'E:/Kaggle/05-LabelEncoded-last/sample_submission.csv'